In [82]:
import importlib
import utils

import os
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
from scipy.stats import mode
import nltk

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from nltk.corpus import stopwords

importlib.reload(utils)
from utils import Preprocess_Tweets

In [47]:
end = datetime.now()
start = datetime(end.year, end.month, end.day-7)

In [48]:
stock = "GOOG"
google_stock = yf.download(stock, start, end)

[*********************100%%**********************]  1 of 1 completed


In [49]:
google_stock.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-07-08,191.365005,191.679001,189.320007,190.479996,190.479996,12097600
2024-07-09,191.750000,192.860001,190.229996,190.440002,190.440002,10198500
2024-07-10,190.750000,193.309998,190.619995,192.660004,192.660004,12052900
2024-07-11,191.339996,192.410004,186.820007,187.300003,187.300003,16452000
2024-07-12,186.919998,188.690002,186.139999,186.779999,186.779999,14429100


In [50]:
tweetsDf = pd.read_csv("tweets.csv")
tweetsDf.head(15)

,Date,Tweet
0,Tue Jan 02 21:09:21 +0000 2018,Good closing #price on $GE today. Looking forw...
1,Sun Jan 14 16:19:31 +0000 2018,$GE is a long term #buy at today's #stock pric...
2,Fri Jan 05 21:25:31 +0000 2018,Watch Us Report LIVE from the Floor of the NYS...
3,Tue Jan 16 17:49:14 +0000 2018,Just a Fun Fact on #DJIA RSI on Dow Jones We...
4,Wed Jan 17 08:15:51 +0000 2018,Repeat after me: the 👏🏼 Dow 👏🏼 Jones 👏🏼 doesn’...
5,Fri Jan 05 21:23:49 +0000 2018,Watch Us Report LIVE from the Floor of the NYS...
6,Wed Jan 31 14:40:47 +0000 2018,Which Way Wednesday - Fed Edition $AAPL $DIA #...
7,Thu Jan 25 16:15:23 +0000 2018,Greater Fool Theory: buy a stock at an outrage...
8,Fri Jan 26 19:14:08 +0000 2018,How to follow Stock Price Movement for Profit....
9,Sat Jan 27 18:42:32 +0000 2018,Notable Earnings 📈📉 M $IDTI $LMT $RMBS Tue $...


In [51]:
# Convert the Date column to datetime
tweetsDf['Date'] = pd.to_datetime(tweetsDf['Date'], format='%a %b %d %H:%M:%S %z %Y')

# Format the Date column to the desired format
tweetsDf['Date'] = tweetsDf['Date'].dt.strftime('%Y-%m-%d')

tweetsDf.head()

,Date,Tweet
0,2018-01-02,Good closing #price on $GE today. Looking forw...
1,2018-01-14,$GE is a long term #buy at today's #stock pric...
2,2018-01-05,Watch Us Report LIVE from the Floor of the NYS...
3,2018-01-16,Just a Fun Fact on #DJIA RSI on Dow Jones We...
4,2018-01-17,Repeat after me: the 👏🏼 Dow 👏🏼 Jones 👏🏼 doesn’...


In [52]:
Preprocess_Tweets(tweetsDf)

,Date,Tweet,Tweet_Cleaned
0,2018-01-02,Good closing #price on $GE today. Looking forw...,good closing on ge today looking forward to mo...
1,2018-01-14,$GE is a long term #buy at today's #stock pric...,ge is a long term at todays price aapl amzn ba...
2,2018-01-05,Watch Us Report LIVE from the Floor of the NYS...,watch us report live from the floor of the nys...
3,2018-01-16,Just a Fun Fact on #DJIA RSI on Dow Jones We...,just a fun fact on rsi on dow jones weekly cha...
4,2018-01-17,Repeat after me: the 👏🏼 Dow 👏🏼 Jones 👏🏼 doesn’...,repeat after me the 👏🏼 dow 👏🏼 jones 👏🏼 doesn’t...
...,...,...,...
8004,2024-06-13,"Very quiet day again, digestion. $SMH $NVDA $A...",very quiet day again digestion smh nvda avgo m...
8005,2024-06-08,Opp cost is real. While u have been sitting in...,opp cost is real while u have been sitting in ...
8006,2024-06-10,Overall very quiet day. $SMH new ath.. $MU $AR...,overall very quiet day smh new ath mu arm lead...
8007,2024-06-28,"There's no debate about it, the markets did NO...",theres no debate about it the markets did not ...


In [ ]:
print(stopwords.words('english'))

In [86]:
StopWords = set([s.replace("'", "") for s in stopwords.words('english') if s not in ["not", "up", "down", "above", "below", "under", "over"]])

tweetsDf['Tweet_Cleaned'] = tweetsDf['Tweet_Cleaned'].apply(lambda x: ' '.join([word for word in x.split() if word not in StopWords]))
tweetsDf['Tweet_Cleaned'] = tweetsDf['Tweet_Cleaned'].str.strip()

In [53]:
tweetsDf = tweetsDf.drop(['Tweet'], axis=1)

In [55]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [56]:
def sentiment_score(tweet):
  tokens = tokenizer.encode(tweet, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits))+1

In [93]:
tweetsDf['sentiment'] = tweetsDf['Tweet_Cleaned'].apply(lambda x: sentiment_score(x[:512]))

In [96]:
tweetsDf

,Date,Tweet_Cleaned,sentiment
0,2018-01-02,good closing ge today looking forward activity...,4
1,2018-01-14,ge long term todays price aapl amzn baba cost ...,3
2,2018-01-05,watch us report live floor nyse weeks weekly w...,1
3,2018-01-16,fun fact rsi dow jones weekly chart 90.20 high...,5
4,2018-01-17,repeat 👏🏼 dow 👏🏼 jones 👏🏼 doesn’t 👏🏼 indicate ...,5
...,...,...,...
8004,2024-06-13,quiet day digestion smh nvda avgo mu etc conti...,5
8005,2024-06-08,opp cost real u sitting promised bagger stock ...,1
8006,2024-06-10,overall quiet day smh new ath mu arm leading g...,4
8007,2024-06-28,theres debate markets not like nkes report cha...,1


In [108]:
grouped = tweetsDf.groupby('Date')['sentiment'].apply(lambda x: x.mean().round().astype(int))

date_sentiment = pd.DataFrame({'Date': grouped.index, 'SentimentIndicator': grouped.values})

date_sentiment.to_csv('date_sentiment.csv', index=False)

In [110]:
date_sentiment['SentimentIndicator'].value_counts()

SentimentIndicator
2    848
1    641
3    411
4    148
5     96
Name: count, dtype: int64

<p>On a scrapped tweets we made preprocessing converting all tweets in lower case, deleting StopWords, replace punctuations and made some replacements of slang words</p>
<p>After we made tokenization of tweets and predicted sentiment of each tweet</p>
<p>And as the result group all tweets by Date and made SentimentIndicator for each Date to determine main movement of Stock Market</p>